In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [6]:
data1 = pd.read_csv("../../dataset/NASA/KC2.csv")
data1.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,no
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,yes
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47.0,106.0,692.0,467.0,106.0,yes
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23.0,67.0,343.0,232.0,65.0,yes
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18.0,58.0,310.0,190.0,51.0,yes


In [7]:
data1.describe()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
count,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,5.220000e+02,...,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000,522.000000
mean,36.894828,4.893487,2.447126,3.650192,94.630843,555.472644,0.274464,9.735287,28.320441,1.854299e+04,...,1094.811456,27.772031,2.000000,4.339080,0.281609,9.197701,14.465900,57.611494,37.023372,8.765134
std,77.944048,10.976772,6.665003,8.054860,233.230165,1817.468320,0.275609,11.088640,32.225597,1.132712e+05,...,7556.521581,64.431485,5.582052,9.214753,1.038236,6.360180,22.086661,142.990741,90.398620,21.942779
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
25%,4.000000,1.000000,1.000000,1.000000,5.000000,11.610000,0.060000,1.500000,7.740000,1.741000e+01,...,0.970000,2.000000,0.000000,0.000000,0.000000,3.000000,2.000000,4.000000,2.000000,1.000000
50%,13.000000,2.000000,1.000000,2.000000,27.000000,109.205000,0.140000,6.000000,20.560000,6.135900e+02,...,34.090000,8.000000,0.000000,1.000000,0.000000,8.000000,7.000000,16.500000,11.000000,3.000000
75%,45.000000,5.000000,1.000000,4.000000,104.750000,543.787500,0.500000,14.610000,38.365000,7.624085e+03,...,423.562500,33.750000,2.000000,5.000000,0.000000,14.000000,20.000000,64.000000,41.000000,9.000000
max,1275.000000,180.000000,125.000000,143.000000,3982.000000,33814.560000,2.000000,103.530000,415.060000,2.147484e+06,...,153047.010000,1107.000000,44.000000,121.000000,11.000000,47.000000,325.000000,2469.000000,1513.000000,361.000000


In [8]:
data1.values.shape

(522, 22)

In [9]:
data2=data1.values
for i in range(522):
    if data2[i][21]=="yes":
        data2[i][21]=1
    else:
        data2[i][21]=0
data2[1]

array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1, 1,
       1, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1], dtype=object)

In [10]:
X=np.array(data2).astype(float)  

In [11]:
X[:,:21]

array([[  1.1,   1.4,   1.4, ...,   1.2,   1.2,   1.4],
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [415. ,  59. ,  50. , ..., 692. , 467. , 106. ],
       ...,
       [  4. ,   1. ,   1. , ...,   3. ,   1. ,   1. ],
       [  4. ,   1. ,   1. , ...,   3. ,   2. ,   1. ],
       [  3. ,   1. ,   1. , ...,   1. ,   0. ,   1. ]])

In [12]:
y=X[:,21]

In [13]:
X=X[:,0:21]

In [14]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((830, 21), (830,))

In [15]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(830, 21)

In [16]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [17]:
#将数据集划分为训练集和测试集
traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.2,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [18]:
k_acc_list, k_f1_list, k_auc_list, k_mcc_list, k_Gm_list=[],[],[],[],[]
#这里设置shuffle设置为ture就是打乱顺序在分配
kf = KFold(n_splits=10,shuffle=True,random_state=42)
for map_times in np.arange(15,36,5):
    acc_list_tmp, f1_list_tmp, auc_list_tmp, mcc_list_tmp,Gm_list_tmp=[],[],[],[],[]
    for enhance_times in np.arange(15, 36,5):
        for k, (train, test) in enumerate(kf.split(traindata, trainlabel)):
            # kf.split输出的是索引，所以由索引获取交叉后的训练集和测试集及标签
            k_train_data,k_train_label = traindata[train], trainlabel[train]
            k_test_data,k_test_label = traindata[test], trainlabel[test]           

            bls = broadnet(maptimes = map_times, 
                       enhencetimes = enhance_times,
                       map_function = 'relu',
                       enhence_function = 'relu',
                       batchsize =100,
                       reg = 0.001)

            #根据训练数据设置权重
            weight=bls.addWeight(k_train_label)

            #训练
            starttime = datetime.datetime.now()
            bls.fit(k_train_data,k_train_label,weight)
            endtime = datetime.datetime.now()
            #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

            #print('k_test_label:', k_test_label)
            #预测
            k_predict_label = bls.predict(k_test_data)
            #print('k_predict_label:', k_predict_label)

            #评价指标计算
            acc=accuracy_score(k_test_label,k_predict_label,normalize=True)
            fmeasure=f1_score(k_test_label,k_predict_label, average='weighted', labels=np.unique(k_test_label))
            try:
                auc=roc_auc_score(k_test_label,k_predict_label, average='macro', sample_weight=None)
            except ValueError:
                pass
            MCC=matthews_corrcoef(k_test_label,k_predict_label)
            Gmeasure=geometric_mean_score(k_test_label,k_predict_label, average='weighted')

            #将此次的十折交叉验证的结果 (10个)保存到pi_list_tmp中
            acc_list_tmp.append(acc)
            f1_list_tmp.append(fmeasure)
            auc_list_tmp.append(auc)
            mcc_list_tmp.append(MCC)
            Gm_list_tmp.append(Gmeasure)

        #求平均保存到k_acc_list中   
        k_average_acc=np.mean(acc_list_tmp)
        k_average_acc=round(k_average_acc,5)
        k_acc_list.append(k_average_acc)

        #求平均保存到k_f1_list中   
        k_average_f1=np.mean(f1_list_tmp)
        k_average_f1=round(k_average_f1,5)
        k_f1_list.append(k_average_f1)

        #求平均保存到k_auc_list中   
        k_average_auc=np.mean(auc_list_tmp)
        k_average_auc=round(k_average_auc,5)
        k_auc_list.append(k_average_auc)

        #求平均保存到k_mcc_list中   
        k_average_mcc=np.mean(mcc_list_tmp)
        k_average_mcc=round(k_average_mcc,5)
        k_mcc_list.append(k_average_mcc)

        #求平均保存到k_Gm_list中   
        k_average_Gm=np.mean(Gm_list_tmp)
        k_average_Gm=round(k_average_Gm,5)
        k_Gm_list.append(k_average_Gm)
        print(f'maptiems:{map_times}\tenhancetimes:{enhance_times}\tk_average_acc:{k_average_acc}\tk_average_f1:{k_average_f1}\tk_average_auc:{k_average_auc}\tk_average_mcc:{k_average_mcc}\tk_average_Gm:{k_average_Gm}\t')

k_acc_array=np.array(k_acc_list)
k_acc_array=k_acc_array.reshape(5,5)
# 一维最大值索引
#idx_max_ravel = np.argmax(k_acc_array)
# true索引
#idx_max = np.unravel_index(idx_max_ravel, k_acc_array.shape)
#print('max times:',idx_max)

k_f1_array=np.array(k_f1_list)
k_f1_array=k_f1_array.reshape(5,5)

k_auc_array=np.array(k_auc_list)
k_auc_array=k_auc_array.reshape(5,5)

k_mcc_array=np.array(k_mcc_list)
k_mcc_array=k_mcc_array.reshape(5,5)

k_Gm_array=np.array(k_Gm_list)
k_Gm_array=k_Gm_array.reshape(5,5)

sio.savemat('./data_remember/WBLS_Smote_KC2_MEbest.mat',{'acc':k_acc_array,'f1':k_f1_array,'auc':k_auc_array,'mcc':k_mcc_array,'Gm':k_Gm_array})

maptiems:15	enhancetimes:15	k_average_acc:0.77696	k_average_f1:0.77656	k_average_auc:0.77745	k_average_mcc:0.55621	k_average_Gm:0.77744	
maptiems:15	enhancetimes:20	k_average_acc:0.77024	k_average_f1:0.76994	k_average_auc:0.77097	k_average_mcc:0.54334	k_average_Gm:0.77096	
maptiems:15	enhancetimes:25	k_average_acc:0.76242	k_average_f1:0.7618	k_average_auc:0.76306	k_average_mcc:0.52836	k_average_Gm:0.76304	
maptiems:15	enhancetimes:30	k_average_acc:0.7638	k_average_f1:0.76324	k_average_auc:0.76479	k_average_mcc:0.53165	k_average_Gm:0.76478	
maptiems:15	enhancetimes:35	k_average_acc:0.7676	k_average_f1:0.7671	k_average_auc:0.76847	k_average_mcc:0.539	k_average_Gm:0.76846	
maptiems:20	enhancetimes:15	k_average_acc:0.76343	k_average_f1:0.76278	k_average_auc:0.76554	k_average_mcc:0.53286	k_average_Gm:0.76552	
maptiems:20	enhancetimes:20	k_average_acc:0.76348	k_average_f1:0.76289	k_average_auc:0.76508	k_average_mcc:0.53218	k_average_Gm:0.76507	
maptiems:20	enhancetimes:25	k_average_acc:0.770